### TRAINING THE MODEL

Using CNN

In [1]:
import numpy as np
data = np.load('Image_data.npy')
target = np.load('Category.npy')
#loading the numpy array save in the previous(1.0 PROCESSING) notebook

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation

Using TensorFlow backend.


In [3]:
#initializing cnn
model = Sequential()
data.shape

(1549, 100, 100, 1)

In [4]:
#step 1 Convolution

#Conv2D(no. of feature maps you want , filter_size you want, image input size you want)
model.add(Conv2D(200, (3,3), input_shape = data.shape[1:]))
model.add(Activation('relu'))

#step2 MaxPooling
model.add(MaxPooling2D(pool_size = (2,2)))

In [5]:
#adding one more convolutional layer and maxpooling layer to increase the accuracy

model.add(Conv2D(100, (3,3), activation = 'relu' ))
model.add(MaxPooling2D(pool_size = (2,2)))

In [6]:
#step 3 flattening 

model.add(Flatten()) #it will open the matrix and create the inputs for the ann
model.add(Dropout(0.5))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# Step 4 Full Connection
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(2, activation = 'softmax'))

In [8]:
#compiling
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
data.shape

(1549, 100, 100, 1)

In [10]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=.20)

In [11]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('model_fmd', monitor = 'val_loss',verbose = 0, save_best_only = True, mode = 'auto')
hist = model.fit(train_data,train_target, epochs = 20, callbacks = [checkpoint], validation_split = 0.2)




Train on 991 samples, validate on 248 samples
Epoch 1/20





991/991 [==============================] - 59s 60ms/step - loss: 0.6783 - acc: 0.5742 - val_loss: 0.5815 - val_acc: 0.7258
Epoch 2/20
991/991 [==============================] - 56s 57ms/step - loss: 0.4532 - acc: 0.7750 - val_loss: 0.4147 - val_acc: 0.8145
Epoch 3/20
991/991 [==============================] - 57s 58ms/step - loss: 0.3329 - acc: 0.8658 - val_loss: 0.3236 - val_acc: 0.8589
Epoch 4/20
991/991 [==============================] - 58s 58ms/step - loss: 0.2369 - acc: 0.9051 - val_loss: 0.2678 - val_acc: 0.8589
Epoch 5/20
991/991 [==============================] - 56s 57ms/step - loss: 0.1607 - acc: 0.9324 - val_loss: 0.2645 - val_acc: 0.8790
Epoch 6/20
991/991 [==============================] - 55s 56ms/step - loss: 0.1351 - acc: 0.9455 - val_loss: 0.2666 - val_acc: 0.8710
Epoch 7/20
991/991 [==============================] - 58s 58ms/step - loss: 0.1080 - acc: 0.9596 - val_loss: 0.3258 - val_acc: 0.8992
Epoch 8/

In [ ]:
import matplotlib.pyplot as plt

plt.plot( hist.history["loss"], label="train_loss")
plt.plot(hist.history["val_loss"], label="val_loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

In [ ]:
print(model.evaluate(test_data,test_target))